In [1]:
import sys
from Load import *
from UE import *
from Satellite import *
from Oracle import *
import time
import os

def initial_assignment(UEss, Satss, oracle):
    C = UEss[0].coverage_info
    if oracle is None:
        # random assign
        for ueid in UEss:
            ue = UEss[ueid]
            possible_satellites = np.where(C[ueid, :, 0] == 1)[0]
            assert (len(possible_satellites) > 0)
            satid = random.choice(possible_satellites)
            ue.serving_satellite = Satss[satid]
            ue.serving_satellite_history.append(satid)
            serving_length = ue.estimate_serving_length(satid)
            Satss[satid].increment_my_load(serving_length, UE_HANDOVER_SIGNALLING_COUNT_ON_SOURCE)
    else:
        for ueid in UEss:
            ue = UEss[ueid]
            satid = oracle.query_init_satellite(ueid)
            assert (C[ueid, satid, 0] == 1)
            ue.serving_satellite = Satss[satid]
            ue.serving_satellite_history.append(satid)


def monitor_timestamp(env):
    while True:
        if env.now % 100 == 0:
            print(f"Current simulation time {env.now}", file=sys.stderr)
        yield env.timeout(1)


if __name__ == "__main__":
    SOURCE_ALG = sys.argv[1]
    CANDIDATE_ALG = sys.argv[2]
    # configuration
    # When oracle_simulation is False, the oracle_assignment could be True/False
    # When oracle_simulation is True, the oracle_assignment must be True
    if oracle_simulation:
        assert (oracle_assignment)
    random.seed(10)

    # Loading scenarios
    beginning = time.time()
    UEs_template, satellites_template, C = load_scenario()
    DURATION = C.shape[2]
    env = simpy.Environment()

    assignment_oracle = None
    simulation_oracle = None
    if oracle_assignment:
        assignment_oracle = Oracle()
    if oracle_simulation:
        simulation_oracle = assignment_oracle

    UEs = {}
    for ue_template in UEs_template:
        ID = ue_template.ID
        if ID not in UEs:
            UEs[ID] = UE(
                identity=ID,
                position_x=ue_template.x,
                position_y=ue_template.y,
                coverage_info=C,
                oracle=simulation_oracle,
                env=env
            )
        else:
            print("ERROR")

    Satellites = {}
    for sat_template in satellites_template:
        ID = sat_template.ID
        if ID not in Satellites:
            Satellites[ID] = Satellite(
                identity=ID,
                position_x=sat_template.x,
                position_y=sat_template.y,
                height=sat_template.h,
                coverage_r=sat_template.r,
                velocity=sat_template.v,
                sind=sat_template.sind,
                cosd=sat_template.cosd,
                coverage_info=C,
                max_access_opportunity=max_access_opportunity,
                max_access_slots=max_access_slots,
                env=env,
                oracle=simulation_oracle,
            )
        else:
            print("ERROR")

    for ueid in UEs:
        UEs[ueid].satellites = Satellites

    for satid in Satellites:
        Satellites[satid].UEs = UEs
        Satellites[satid].satellites = Satellites

    print(f"Loading scenario in the simulation takes {time.time() - beginning}")
    # ========= UE, SAT objectives are ready =========
    initial_assignment(UEs, Satellites, assignment_oracle)

    env.process(monitor_timestamp(env))

    print('============= Experiment Log =============')
    env.run(until=DURATION)
    print('============= Experiment Ends =============')
    # counter = allCounters(Satellites, UEs)
    # counter.give_result()


Loading scenario in the simulation takes 0.09161114692687988
============= Experiment Log =============
[13] UE 6 sends Satellite 1 the message {"task": "MEASUREMENT_REPORT", "candidates": [0, 2, 3, 4, 7], "utility": [1237, 3062, 1127, 3310, 3234], "from": 6, "to": 1}
[13] Satellite 1 sends Satellite 0 the message {"task": "HANDOVER_REQUEST", "ueid": 6, "candidates": [0, 4, 7], "utility": [1237, 3310, 3234], "priority_load": [0, [5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], "candidates_priority_load": [[0, [], []], [0, [], []], [0, [], []]], "from": 1, "to": 0}
[13] Satellite 1 sends Satellite 4 the message {"task": "HANDOVER_REQUEST", "ueid": 6, "candidates": [0, 4, 7], "utility": [1237, 3310, 3234], "priority_load": [0, [5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], "candidates_priority_load": [[0, [], []], [0, [], []], [0, [], []]], "from": 1, "to": 4}
[13] Satellite 1 sends Satellite 7 the message {"task": "HANDOVER_REQUEST", "ueid":

Current simulation time 0
Current simulation time 100
Current simulation time 200
Current simulation time 300
Current simulation time 400
Current simulation time 500
Current simulation time 600
Current simulation time 700
Current simulation time 800
Current simulation time 900
Current simulation time 1000
Current simulation time 1100


[746] UE 8 sends Satellite 3 the message {"task": "MEASUREMENT_REPORT", "candidates": [0, 1, 2, 4, 5, 7], "utility": [1469, 151, 3165, 3042, 4534, 3386], "from": 8, "to": 3}
[746] Satellite 3 sends Satellite 2 the message {"task": "HANDOVER_REQUEST", "ueid": 8, "candidates": [2, 4, 5], "utility": [3165, 3042, 4534], "priority_load": [0, [5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], "candidates_priority_load": [[0, [], []], [0, [], []], [0, [], []]], "from": 3, "to": 2}
[746] Satellite 3 sends Satellite 4 the message {"task": "HANDOVER_REQUEST", "ueid": 8, "candidates": [2, 4, 5], "utility": [3165, 3042, 4534], "priority_load": [0, [5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], "candidates_priority_load": [[0, [], []], [0, [], []], [0, [], []]], "from": 3, "to": 4}
[746] Satellite 3 sends Satellite 5 the message {"task": "HANDOVER_REQUEST", "ueid": 8, "candidates": [2, 4, 5], "utility": [3165, 3042, 4534], "priority_load": [0, [5, 0, 0, 0

Current simulation time 1200
Current simulation time 1300
Current simulation time 1400
Current simulation time 1500
Current simulation time 1600
Current simulation time 1700
Current simulation time 1800
Current simulation time 1900
Current simulation time 2000
Current simulation time 2100
Current simulation time 2200
Current simulation time 2300


[2192] UE 16 sends Satellite 0 the message {"task": "MEASUREMENT_REPORT", "candidates": [2, 4, 5, 6, 7, 8, 9], "utility": [1708, 1582, 3063, 3235, 1925, 5038, 1921], "from": 16, "to": 0}
[2192] Satellite 0 sends Satellite 6 the message {"task": "HANDOVER_REQUEST", "ueid": 16, "candidates": [6, 2, 7], "utility": [3235, 1708, 1925], "priority_load": [0, [5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], "candidates_priority_load": [[2, [], []], [0, [], []], [2, [], []]], "from": 0, "to": 6}
[2192] Satellite 0 sends Satellite 2 the message {"task": "HANDOVER_REQUEST", "ueid": 16, "candidates": [6, 2, 7], "utility": [3235, 1708, 1925], "priority_load": [0, [5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], "candidates_priority_load": [[2, [], []], [0, [], []], [2, [], []]], "from": 0, "to": 2}
[2192] Satellite 0 sends Satellite 7 the message {"task": "HANDOVER_REQUEST", "ueid": 16, "candidates": [6, 2, 7], "utility": [3235, 1708, 1925], "priority_loa

...

Current simulation time 16700
Current simulation time 16800
Current simulation time 16900
Current simulation time 17000
Current simulation time 17100
Current simulation time 17200
Current simulation time 17300
Current simulation time 17400
Current simulation time 17500
Current simulation time 17600
Current simulation time 17700
Current simulation time 17800
Current simulation time 17900
Current simulation time 18000


[17628] UE 6 sends Satellite 29 the message {"task": "MEASUREMENT_REPORT", "candidates": [32, 33, 34, 36, 37, 39], "utility": [2096, 161, 4000, 4000, 1791, 4000], "from": 6, "to": 29}
[17628] Satellite 29 sends Satellite 39 the message {"task": "HANDOVER_REQUEST", "ueid": 6, "candidates": [39, 36, 37], "utility": [4000, 4000, 1791], "priority_load": [0, [5, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], "candidates_priority_load": [[0, [], []], [2, [], []], [0, [], []]], "from": 29, "to": 39}
[17628] Satellite 29 sends Satellite 36 the message {"task": "HANDOVER_REQUEST", "ueid": 6, "candidates": [39, 36, 37], "utility": [4000, 4000, 1791], "priority_load": [0, [5, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], "candidates_priority_load": [[0, [], []], [2, [], []], [0, [], []]], "from": 29, "to": 36}
[17628] Satellite 29 sends Satellite 37 the message {"task": "HANDOVER_REQUEST", "ueid": 6, "candidates": [39, 36, 37], "utility": [4000, 4000, 1791

Current simulation time 18100
Current simulation time 18200
Current simulation time 18300
Current simulation time 18400
Current simulation time 18500
Current simulation time 18600
Current simulation time 18700
Current simulation time 18800
Current simulation time 18900
Current simulation time 19000
Current simulation time 19100
Current simulation time 19200
Current simulation time 19300
Current simulation time 19400
Current simulation time 19500
Current simulation time 19600


[18971] UE 8 sends Satellite 29 the message {"task": "MEASUREMENT_REPORT", "candidates": [32, 34, 35, 36, 37, 38, 39], "utility": [4000, 4000, 4000, 4000, 4000, 4000, 4000], "from": 8, "to": 29}
[18971] Satellite 29 sends Satellite 38 the message {"task": "HANDOVER_REQUEST", "ueid": 8, "candidates": [38, 35, 36], "utility": [4000, 4000, 4000], "priority_load": [0, [5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], "candidates_priority_load": [[0, [], []], [0, [], []], [2, [], []]], "from": 29, "to": 38}
[18971] Satellite 29 sends Satellite 35 the message {"task": "HANDOVER_REQUEST", "ueid": 8, "candidates": [38, 35, 36], "utility": [4000, 4000, 4000], "priority_load": [0, [5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], "candidates_priority_load": [[0, [], []], [0, [], []], [2, [], []]], "from": 29, "to": 35}
[18971] Satellite 29 sends Satellite 36 the message {"task": "HANDOVER_REQUEST", "ueid": 8, "candidates": [38, 35, 36], "utility": [4000,

Current simulation time 19700
Current simulation time 19800
Current simulation time 19900
